In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:


# Load MNIST data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(f"Original shapes - x_train: {x_train.shape}, x_test: {x_test.shape}")

In [ ]:
def RBF(x, c, s):
    """Radial Basis Function with proper broadcasting"""
    # Calculate squared Euclidean distance
    diff = x - c
    squared_distance = np.sum(diff ** 2)
    return np.exp(-squared_distance / (2 * s ** 2))

In [ ]:
# Convert 28×28 images to 32×32 using RBF interpolation
def convert_to_32x32_rbf(image_28x28, sigma=1.0):
    """Convert 28x28 image to 32x32 using RBF interpolation"""
    # Create a 32x32 grid
    new_image = np.zeros((32, 32))
    
    # Create coordinate grids
    old_coords = np.linspace(0, 31, 28)  # Map 28 points to 0-31 range
    new_coords = np.arange(32)           # Full 32x32 grid
    
    # For each position in the new 32x32 grid
    for i in range(32):
        for j in range(32):
            # Find weights based on distance to original points
            weights = []
            values = []
            
            for oi, old_i in enumerate(old_coords):
                for oj, old_j in enumerate(old_coords):
                    # Calculate RBF weight based on distance
                    distance = np.sqrt((i - old_i)**2 + (j - old_j)**2)
                    weight = np.exp(-distance**2 / (2 * sigma**2))
                    weights.append(weight)
                    values.append(image_28x28[oi, oj])
            
            # Weighted average
            weights = np.array(weights)
            values = np.array(values)
            if np.sum(weights) > 0:
                new_image[i, j] = np.sum(weights * values) / np.sum(weights)
    
    return new_image

In [ ]:
print("Converting training data...")
x_train_rbf = np.zeros((x_train.shape[0], 32, 32))
for i in range(x_train.shape[0]):
    if i % 10000 == 0:
        print(f"Processing training sample {i}/{x_train.shape[0]}")
    x_train_rbf[i] = convert_to_32x32_rbf(x_train[i])

print("Converting test data...")
x_test_rbf = np.zeros((x_test.shape[0], 32, 32))
for i in range(x_test.shape[0]):
    if i % 2000 == 0:
        print(f"Processing test sample {i}/{x_test.shape[0]}")
    x_test_rbf[i] = convert_to_32x32_rbf(x_test[i])

In [ ]:



# Split dataset: 80% training, 10% validation, 10% test
train_size = int(0.8 * x_train_rbf.shape[0])
val_size = int(0.1 * x_train_rbf.shape[0])

x_train_final = x_train_rbf[:train_size]
y_train_final = y_train[:train_size]

x_val_rbf = x_train_rbf[train_size:train_size + val_size]
y_val = y_train[train_size:train_size + val_size]

# Keep only 10% of test data to match the requirement
test_size = int(0.1 * x_test_rbf.shape[0])
x_test_final = x_test_rbf[:test_size]
y_test_final = y_test[:test_size]

print(f"Final dataset shapes:")
print(f"Training: {x_train_final.shape}, {y_train_final.shape}")
print(f"Validation: {x_val_rbf.shape}, {y_val.shape}")
print(f"Test: {x_test_final.shape}, {y_test_final.shape}")

In [ ]:

# Visualize original vs RBF converted images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i in range(5):
    # Original 28x28
    axes[0, i].imshow(x_train[i], cmap='gray')
    axes[0, i].set_title(f'Original 28x28 - Label: {y_train[i]}')
    axes[0, i].axis('off')
    
    # RBF converted 32x32
    axes[1, i].imshow(x_train_rbf[i], cmap='gray')
    axes[1, i].set_title(f'RBF 32x32 - Label: {y_train[i]}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def create_model(hidden_layers, hidden_neurons, activation, optimizer, loss):
    """Create a fully connected neural network model"""
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(32, 32)))
    
    # Add hidden layers according to the number specified
    for i in range(hidden_layers):
        if i < len(hidden_neurons):  # Ensure we don't go out of bounds
            model.add(tf.keras.layers.Dense(hidden_neurons[i], activation=activation))
    
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Define hyperparameter combinations
configurations = [
    # Configuration 1: 1 hidden layer with 16 neurons
    {'hidden_layers': 1, 'hidden_neurons': [16], 'activation': 'sigmoid'},
    # Configuration 2: 2 hidden layers with 16, 32 neurons
    {'hidden_layers': 2, 'hidden_neurons': [16, 32], 'activation': 'sigmoid'},
    # Configuration 3: 3 hidden layers with 16, 32, 64 neurons
    {'hidden_layers': 3, 'hidden_neurons': [16, 32, 64], 'activation': 'sigmoid'}
]

optimizer_loss_combinations = [
    # Gradient Descent with Squared Error Loss
    {'optimizer': tf.keras.optimizers.SGD(learning_rate=0.001), 
     'loss': 'mean_squared_error', 
     'optimizer_name': 'SGD'},
    # Adam with Categorical Cross Entropy
    {'optimizer': tf.keras.optimizers.Adam(learning_rate=0.001), 
     'loss': 'categorical_crossentropy', 
     'optimizer_name': 'Adam'}
]

results = []

In [ ]:
print("\nStarting training for all combinations...")
print("="*80)

for config in configurations:
    for opt_loss in optimizer_loss_combinations:
        print(f"\nConfiguration: {config['hidden_layers']} layers {config['hidden_neurons']}")
        print(f"Optimizer: {opt_loss['optimizer_name']}, Loss: {opt_loss['loss']}")
        print("-" * 50)
        
        # Create and train model
        model = create_model(
            config['hidden_layers'], 
            config['hidden_neurons'], 
            config['activation'],
            opt_loss['optimizer'], 
            opt_loss['loss']
        )
        
        # Convert labels based on loss function
        if opt_loss['loss'] == 'categorical_crossentropy':
            y_train_encoded = tf.keras.utils.to_categorical(y_train_final, 10)
            y_val_encoded = tf.keras.utils.to_categorical(y_val, 10)
            y_test_encoded = tf.keras.utils.to_categorical(y_test_final, 10)
        else:  # mean_squared_error
            y_train_encoded = tf.keras.utils.to_categorical(y_train_final, 10)
            y_val_encoded = tf.keras.utils.to_categorical(y_val, 10)
            y_test_encoded = tf.keras.utils.to_categorical(y_test_final, 10)
        
        # Train the model
        history = model.fit(
            x_train_final, y_train_encoded, 
            epochs=10, 
            validation_data=(x_val_rbf, y_val_encoded),
            verbose=1
        )
        
        # Evaluate on test data
        test_loss, test_acc = model.evaluate(x_test_final, y_test_encoded, verbose=0)
        
        # Store results
        result = {
            'hidden_layers': config['hidden_layers'],
            'hidden_neurons': config['hidden_neurons'],
            'activation': config['activation'],
            'optimizer': opt_loss['optimizer_name'],
            'loss': opt_loss['loss'],
            'test_accuracy': test_acc,
            'test_loss': test_loss
        }
        results.append(result)
        
        print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")
        print("="*80)


Original shapes - x_train: (60000, 28, 28), x_test: (10000, 28, 28)
Converting training data...
Processing training sample 0/60000


In [ ]:
# Display summary of all results
print("\nSUMMARY OF ALL RESULTS:")
print("="*100)
print(f"{'Config':<15} {'Neurons':<15} {'Optimizer':<10} {'Loss':<20} {'Test Acc':<10} {'Test Loss':<10}")
print("-"*100)

for result in results:
    neurons_str = str(result['hidden_neurons'])
    config_str = f"{result['hidden_layers']} layers"
    
    print(f"{config_str:<15} {neurons_str:<15} {result['optimizer']:<10} {result['loss']:<20} "
          f"{result['test_accuracy']:<10.4f} {result['test_loss']:<10.4f}")

# Find best performing model
best_result = max(results, key=lambda x: x['test_accuracy'])
print(f"\nBEST PERFORMING MODEL:")
print(f"Configuration: {best_result['hidden_layers']} layers {best_result['hidden_neurons']}")
print(f"Optimizer: {best_result['optimizer']}, Loss: {best_result['loss']}")
print(f"Test Accuracy: {best_result['test_accuracy']:.4f}")
print(f"Test Loss: {best_result['test_loss']:.4f}")